# Installation

In [ ]:
# pip install -r requirements.txt

In [19]:
!nvidia-smi

Sat Feb  8 14:58:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.03                 Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
| 44%   49C    P0             47W /  170W |    1399MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
pip install faiss-cpu faiss-gpu-cu12

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu-cu12 (from versions: none)
ERROR: No matching distribution found for faiss-gpu-cu12


# AI Agent Development

**Loading the environment variables (from .env file):**

- USER_AGENT
- OPENAI_API_KEY
- TAVILY_API_KEY
- HUGGINGFACEHUB_API_TOKEN

In [1]:
from dotenv import load_dotenv

load_dotenv()                   # Loading API key from .env file

True

## A) Tools

### 1.Retriever Tool

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS

Document:

In [3]:
loader = WebBaseLoader("https://www.dasa.org")
docs = loader.load()

print(docs[0].metadata['source'])
print(docs[0].metadata['title'])
print(docs[0].metadata['description'])

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
print("# Documents:",len(documents))

https://www.dasa.org
DASA
Transform Your Enterprise Into a High-Performance Digital Organization Tools and expertise to create agile, resilient software teams that drive lasting digital
# Documents: 2


#### - Embedding

Open AI

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()

HuggingFace

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

# Use an open-source embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
vector = FAISS.from_documents(documents, embedding_model)
retriever = vector.as_retriever()
retriever_tool = create_retriever_tool (retriever, "DASA_search", "Search for information about DASA. For any questions about DASA, you must use this tool")

### 2. Web Search Tool

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()

## B) AI Agent

In [7]:
tools = [retriever_tool, search_tool]

### 1. Prompt Template

#### - OpenAI Prompt Template:

- LangChain Hub allows access to predefined prompts, chains, and agents.
- The pulled prompt template designed for OpenAI's function calling capabilities. It is typically formatted for use with OpenAI GPT models in an agent-based setup

In [7]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt

c:\Users\Admin\Desktop\AIPoint Projects\AI Agent Development\.venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

#### - Custome Prompt Template

For using the open-sourced llms.

In [8]:
from langchain.prompts import PromptTemplate

instruction_template = PromptTemplate(
    input_variables=["input"],
    template=(
        "<s>[INST] <<SYS>>\nYou are a helpful AI assistant. Follow the instructions carefully.\n<</SYS>>\n"
        "{input} [/INST]"
    )
)

# Test the template
formatted_prompt = instruction_template.format(input="Explain machine learning in simple terms.")
print(formatted_prompt)


<s>[INST] <<SYS>>
You are a helpful AI assistant. Follow the instructions carefully.
<</SYS>>
Explain machine learning in simple terms. [/INST]


### 2. Loading the LLM

#### - OpenAI LLMs

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model =  "",api_key="Your API Key here")

#### - Open-source LLMs

##### -- Running Locally

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "mistralai/Mistral-7B-Instruct"  # Replace with your preferred model

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Create text generation pipeline
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256
)

# Integrate with LangChain
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

#### -- Using the API

In [10]:
from langchain_community.llms import HuggingFaceHub

# Load Mistral via Hugging Face Inference API
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature": 0.7, "max_new_tokens": 256}
)

### 3. Creating the Agent

#### - OpenAI-based Agent

In [9]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

agent = create_openai_functions_agent(llm, tools ,prompt)
agent_executor=AgentExecutor(agent = agent, tools = tools, verbose = True)

#### -Open-source Agent

In [11]:
from langchain.agents import initialize_agent, AgentType

# Initialize Agent with Mistral/Qwen/Falcon
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_14324\1136816465.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### 4. Running the Agent

### - OpenAI-based

In [ ]:
agent_executor.invoke({"input":"What is the mission of DASA?"})

### - Open-source based

In [12]:
# Test the agent
agent.run("What is the mission of DASA?")

C:\Users\Admin\AppData\Local\Temp\ipykernel_14324\1293561915.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is the mission of DASA?")




> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'page_content', 'input'}